In [1]:
!pip install langchain-text-splitters
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 747.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 11.1 MB/s eta 0:00:00


# load nghị định 100-2019 NĐ-CP

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [21]:
file_path = '/content/Nghị định-100-2019-NĐ-CP.docx.txt'
with open(file_path, 'r') as file:
            content = file.read()

# split content to chunk

In [43]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=30,
    length_function=len,
    is_separator_regex=False,
)

In [44]:
chunks_ = text_splitter.create_documents([content])
chunks = [c.page_content for c in chunks_]
print(f"The text has been broken down in {len(chunks)} chunks.")

The text has been broken down in 103 chunks.


# convert chunk to vector
I use dangvantuan/vietnamese-embedding-LongContext model for embedding model.

In [45]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

sentences = ['That is a happy person', 'That is a very happy person']

embedding_model = SentenceTransformer('dangvantuan/vietnamese-embedding-LongContext',
                                      trust_remote_code=True).cuda()
embeddings = embedding_model.encode(sentences)
print(cos_sim(embeddings[0], embeddings[1]))

tensor([[0.9605]])


In [46]:
embeddings.shape

(2, 768)

In [47]:
import torch
import numpy as np

embeddings_ = []
batch_size = 16

for _id in range(0, len(chunks)//batch_size + int(len(chunks) % batch_size != 0)):
    start_id = _id * batch_size
    end_id = min(len(chunks), start_id + batch_size)
    print(start_id, end_id)
    batch_chunks = chunks[start_id:end_id]
    with torch.no_grad():
        embeddings = embedding_model.encode(batch_chunks)
    embeddings_.append(embeddings)

embeddings = torch.tensor(np.concatenate(embeddings_, 0))
embeddings.shape

0 16
16 32
32 48
48 64
64 80
80 96
96 103


torch.Size([103, 768])

# load LLM model
I use Qwen 2 7B Instruct

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# convert question to embedding

In [48]:
question = 'vi phạm nồng độ cồn bị phạt bao nhiêu tiền?' #'uống rượu bia khi tham gia giao thông có bị phạt tiền không?'
query_embedding = embedding_model.encode([question])
query_embedding = torch.tensor(query_embedding)
query_embedding.shape

torch.Size([1, 768])

# calulate top 10 similar chunk by cosine

In [49]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
scores = cos(query_embedding.cuda(), embeddings.cuda())

In [50]:
indices = scores.topk(10).indices.cpu().long().numpy()
indices

array([82, 80,  9, 27, 51, 12, 54, 40, 10, 13])

In [51]:
import numpy as np
chunks_ = np.array(chunks)
chunks_[indices]

array(['a) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở;\nb) Không chấp hành yêu cầu kiểm tra về nồng độ cồn, chất kích thích khác mà pháp luật cấm sử dụng của người thi hành công vụ;\nc) Khi làm nhiệm vụ mà trong cơ thể có chất kích thích khác mà pháp luật cấm sử dụng.\n8. Ngoài việc bị phạt tiền, cá nhân thực hiện hành vi vi phạm còn bị áp dụng các hình thức xử phạt bổ sung sau đây:\na) Thực hiện hành vi quy định tại khoản 3, khoản 4 Điều này bị tước quyền sử dụng Giấy phép lái tàu từ 01 tháng đến 03 tháng;\nb) Thực hiện hành vi quy định tại khoản 5 Điều này bị tước quyền sử dụng Giấy phép lái tàu từ 03 tháng đến 05 tháng;\nc) Thực hiện hành vi quy định tại khoản 6 Điều này bị tước quyền sử dụng Giấy phép lái tàu từ 10 tháng đến 12 tháng;\nd) Thực hiện hành vi quy định tại khoản 7 Điều này bị tước quyền sử dụng Giấy phép lái tàu từ 22 tháng đến 24 tháng.\n9. Ngoài việc bị áp dụng hình thức xử phạ

# create question prompt

In [52]:
Contexts = "\n".join(chunks_[indices].tolist())
print(Contexts)

a) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở;
b) Không chấp hành yêu cầu kiểm tra về nồng độ cồn, chất kích thích khác mà pháp luật cấm sử dụng của người thi hành công vụ;
c) Khi làm nhiệm vụ mà trong cơ thể có chất kích thích khác mà pháp luật cấm sử dụng.
8. Ngoài việc bị phạt tiền, cá nhân thực hiện hành vi vi phạm còn bị áp dụng các hình thức xử phạt bổ sung sau đây:
a) Thực hiện hành vi quy định tại khoản 3, khoản 4 Điều này bị tước quyền sử dụng Giấy phép lái tàu từ 01 tháng đến 03 tháng;
b) Thực hiện hành vi quy định tại khoản 5 Điều này bị tước quyền sử dụng Giấy phép lái tàu từ 03 tháng đến 05 tháng;
c) Thực hiện hành vi quy định tại khoản 6 Điều này bị tước quyền sử dụng Giấy phép lái tàu từ 10 tháng đến 12 tháng;
d) Thực hiện hành vi quy định tại khoản 7 Điều này bị tước quyền sử dụng Giấy phép lái tàu từ 22 tháng đến 24 tháng.
9. Ngoài việc bị áp dụng hình thức xử phạt, cá nhân thực 

In [53]:
question_prompt = f'''Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.

Context:
{Contexts}

Now here is the question you need to answer.

Question: {question}'''

In [54]:
print(question_prompt)

Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.

Context:
a) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở;
b) Không chấp hành yêu cầu kiểm tra về nồng độ cồn, chất kích thích khác mà pháp luật cấm sử dụng của người thi hành công vụ;
c) Khi làm nhiệm vụ mà trong cơ thể có chất kích thích khác mà pháp luật cấm sử dụng.
8. Ngoài việc bị phạt tiền, cá nhân thực hiện hành vi vi phạm còn bị áp dụng các hình thức xử phạt bổ sung sau đây:
a) Thực hiện hành vi quy định tại khoản 3, khoản 4 Điều này bị tước quyền sử dụng Giấy phép lái tàu từ 01 tháng đến 03 tháng;
b) Thực hiện hành vi quy định tại khoản 5 Điều này bị tước quyền sử dụng Giấy phép

# Ask Qwen2 7B

In [ ]:
device = "cuda" # the device to load the model onto

messages = [
    {"role": "system", "content": "You are a helpful assistant. Please answer question more detail base on context!"},
    {"role": "user", "content": question_prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=2048
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

# Ask Qwen2 72B
Because I can't run it on Colab Free tier, I use Together AI to test Qwen2 72B

In [17]:
pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.1 MB/s eta 0:00:00


In [55]:
import os
from google.colab import userdata
from together import Together

client = Together(api_key=userdata.get('togetherai_api'))

response = client.chat.completions.create(
    model="Qwen/Qwen2-72B-Instruct",
    messages=[{"role": "user", "content": question_prompt}],
)
print(response.choices[0].message.content)

Câu hỏi không rõ ràng vì mức phạt phụ thuộc vào nồng độ cồn cụ thể. Tuy nhiên, dựa trên thông tin đã cho:

- Nếu nồng độ cồn trong máu hoặc hơi thở từ 50 miligam đến 80 miligam/100 mililít máu hoặc từ 0,25 miligam đến 0,4 miligam/1 lít khí thở, mức phạt từ 4.000.000 đồng đến 6.000.000 đồng (Điều 63, khoản 2).

- Nếu nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở, mức phạt từ 6.000.000 đồng đến 8.000.000 đồng (Điều 63, khoản 3a).


# Ask Llama 3.1 405B
Because I can't run it on Colab Free tier, I use Together AI to test Llama 3.1 405B

In [56]:
import os
from google.colab import userdata
from together import Together

client = Together(api_key=userdata.get('togetherai_api'))

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
    messages=[{"role": "user", "content": question_prompt}],
)
print(response.choices[0].message.content)

Điều 6. Xử phạt người điều khiển xe mô tô, xe gắn máy (kể cả xe máy điện), các loại xe tương tự xe mô tô và các loại xe tương tự xe gắn máy vi phạm quy tắc giao thông đường bộ

8. Phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:
e) Điều khiển xe trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở;

Điều 6. Xử phạt người điều khiển xe mô tô, xe gắn máy (kể cả xe máy điện), các loại xe tương tự xe mô tô và các loại xe tương tự xe gắn máy vi phạm quy tắc giao thông đường bộ

7. Phạt tiền từ 4.000.000 đồng đến 5.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:
c) Điều khiển xe trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở.

Điều 6. Xử phạt người điều khiển xe mô tô, xe gắn máy (kể cả xe má

# The result base on LLM model and embedding model.
## I can't run Qwen2 with colab T4.

## Llama 3.1 405B have correct answer!

## Result better than short context because LLM can see contexts with more detail instead of list of random short context (can't link enough information)

## The result still not good enough. I think basic chunking can't create correct contexts. We need try other chunking strategy.

## Spelling errors in the text may occur when downloading as a .txt file. Perhaps we need to clean the data for better results.